In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,70069
2,Huelva,Confirmados PDIA 14 días,8441
3,Huelva,Tasa PDIA 14 días,"1644,87401835649"
4,Huelva,Confirmados PDIA 7 días,3754
5,Huelva,Tasa PDIA 7 dias,"731,5314613091178"
6,Huelva,Total Confirmados,70303
7,Huelva,Curados,53679
8,Huelva,Fallecidos,435


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  70069.0


/tmp/ipykernel_49087/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  20489.0


/tmp/ipykernel_49087/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_49087/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_49087/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_49087/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 131 personas en los últimos 7 días 

Un positivo PCR cada 55 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,70069.0,8441.0,3754.0,513170.0,731.531461,1644.874018,1613.0
Huelva-Costa,40755.0,4507.0,1919.0,289548.0,662.757125,1556.564024,738.0
Sierra de Huelva-Andévalo Central,8719.0,2072.0,1012.0,67391.0,1501.684201,3074.594530,592.0
Huelva (capital),20489.0,2630.0,1099.0,143837.0,764.059317,1828.458602,425.0
Condado-Campiña,20128.0,1848.0,816.0,156231.0,522.303512,1182.863836,279.0
Valverde del Camino,1902.0,427.0,206.0,12750.0,1615.686275,3349.019608,111.0
Aljaraque,2795.0,400.0,171.0,21474.0,796.311819,1862.717705,76.0
Aracena,1300.0,197.0,106.0,8255.0,1284.070260,2386.432465,71.0
Nerva,608.0,213.0,112.0,5169.0,2166.763397,4120.719675,54.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cerro de Andévalo (El),423.0,143.0,63.0,2327.0,2707.348517,6145.251397,43.0
Jabugo,266.0,96.0,50.0,2260.0,2212.389381,4247.787611,36.0
Nerva,608.0,213.0,112.0,5169.0,2166.763397,4120.719675,54.0
Fuenteheridos,80.0,29.0,11.0,709.0,1551.480959,4090.267983,4.0
Cortegana,640.0,181.0,97.0,4602.0,2107.779226,3933.072577,49.0
Rosal de la Frontera,224.0,65.0,36.0,1697.0,2121.390689,3830.288745,19.0
Campillo (El),239.0,76.0,35.0,2024.0,1729.249012,3754.940711,25.0
Cumbres Mayores,291.0,63.0,26.0,1749.0,1486.563751,3602.058319,8.0
Valverde del Camino,1902.0,427.0,206.0,12750.0,1615.686275,3349.019608,111.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Higuera de la Sierra,120.0,14.0,1.0,1291.0,77.459334,1084.430674,1.0,0.071429
Villalba del Alcor,566.0,20.0,4.0,3366.0,118.835413,594.177065,1.0,0.200000
Campofrío,100.0,9.0,2.0,713.0,280.504909,1262.272090,1.0,0.222222
Cortelazor,27.0,7.0,2.0,299.0,668.896321,2341.137124,0.0,0.285714
Villablanca,361.0,20.0,6.0,2885.0,207.972270,693.240901,0.0,0.300000
Niebla,593.0,97.0,30.0,4116.0,728.862974,2356.656948,13.0,0.309278
Paymogo,120.0,19.0,6.0,1162.0,516.351119,1635.111876,1.0,0.315789
Rociana del Condado,915.0,78.0,25.0,7939.0,314.901121,982.491498,9.0,0.320513
Santa Ana la Real,35.0,3.0,1.0,475.0,210.526316,631.578947,0.0,0.333333
